# PyMySQL을 활용한 데이터 입출력 구현

In [16]:
import pymysql
from pandas import DataFrame

# 접속정보 변수 초기화
HOSTNAME = 'localhost'
PORT = 3306
USERNAME = 'root'
PASSWORD = '123qwe!@#'
DATABASE = 'myschool'
CHARSET = 'utf8'

# 데이터베이스 접속하기
dbcon = pymysql.connect(host = HOSTNAME, port = PORT, user = USERNAME, password = PASSWORD,
                       db=DATABASE, charset = CHARSET)

## 데이터 입력, 수정, 삭제

### 1) 커서 객체 생성
<li> SQL을 실행하는 기능을 갖는 커서 객체를 생성하고, execute()함수를 사용하여 SQL문을 파라미터로 전달한다.</li>

In [17]:
# 커서 생성하기
cursor = dbcon.cursor()

### 2) 데이터 저장하기
<li>테이블의 데이터에 상관없이 <b>%s</b>로 정의, execute()함수의 두 번째 파라미터로 이스케이프 문자를 치환하기 위한 튜플을 전달</li>
<li>INSERT문의 경우 sql 수행 후 AUTO_INCREMENT로 지정된 PK값을 반환받을 수 있다.</li>
<li>입력, 수정, 삭제 처리의 경우 dbcon.commit()을 호출해야 저장</li>

In [18]:
# 저장할 데이터 설정
dname = 'Python'
loc = '1401호',

# 저장을 위한 SQL 실행(문자열 데이터도 홑따옴표 사용 안함)
sql = "INSERT INTO department_2 (dname, loc) VALUES (%s, %s)"
cursor.execute(sql, (dname,loc))

# insert시에 생성된 PK값 얻기
# -> 반드시 commit() 전에 조회 (update, delete는 불필요)

new_deptno = dbcon.insert_id()
print('생성된 학과번호 : %d' % new_deptno)

# 변경사항 저장하기
dbcon.commit()

생성된 학과번호 : 204


### 3) 데이터 수정하기
<li>앞에서 저장한 데이터의 학과명과 위치 수정</li>
<li>SQL에 지정되는 모든 변수는 %s 타입 숫자면 str()사용</li>

In [19]:
# 수정할 데이터 설정
dname = '빅데이터'
loc = '공학관'

# 수정을 위한 SQL 실행
sql = "UPDATE department_2 SET dname=%s, loc=%s WHERE deptno=%s"
edit_count = cursor.execute(sql, (dname, loc, new_deptno))

print('수정된 데이터 수 : %d' % edit_count )

dbcon.commit()

수정된 데이터 수 : 1


### 4) 데이터 삭제하기

In [20]:
# 삭제를 위한 sql 실행
sql = 'DELETE FROM department_2 WHERE deptno=%s'
delete_count = cursor.execute(sql, (new_deptno))

print('삭제된 데이터 수 : %d ' % delete_count)

dbcon.commit()

삭제된 데이터 수 : 1 


### 5) 데이터 조회하기
<li>cursor 객체의 fetchone() 함수를 호출하면 조회 결과를 리턴</li>


In [24]:
# 단일행 조회
# 조회를 위한 sql 형식
sql = """select deptno '학과번호', dname '학과이름', loc '학과위치'
from department_2 where deptno=%s"""

# %s에 202라는 값을 적용하여 실행
cursor.execute(sql, (202))

# 조회 결과 한 줄을 튜플로 반환
result = cursor.fetchone()
result

(202, '기계공학과', '4호관')

In [25]:
# 다중행 데이터 조회
# fetchall() 함수 사용
sql = 'select deptno, dname, loc from department_2'
cursor.execute(sql)

result = cursor.fetchall()
result

((101, '컴퓨터공학과', '1호관'),
 (102, '멀티미디어학과', '2호관'),
 (201, '전자공학과', '3호관'),
 (202, '기계공학과', '4호관'))

### 데이터 결과를 딕셔너리로 반환


In [26]:
# 딕셔너리를 반환받기 위한 커서 객체 생성
cursor = dbcon.cursor(pymysql.cursors.DictCursor)

In [27]:
# 단일행 조회
# 조회를 위한 sql 형식
sql = """select deptno '학과번호', dname '학과이름', loc '학과위치'
from department_2 where deptno=%s"""

# %s에 202라는 값을 적용하여 실행
cursor.execute(sql, (202))

# 조회 결과 한 줄을 딕셔너리로 반환
result = cursor.fetchone()
result

{'학과번호': 202, '학과이름': '기계공학과', '학과위치': '4호관'}

In [28]:
# 다중행 데이터 조회
# fetchall() 함수 사용
sql = 'select deptno, dname, loc from department_2'
cursor.execute(sql)

result = cursor.fetchall()
result

[{'deptno': 101, 'dname': '컴퓨터공학과', 'loc': '1호관'},
 {'deptno': 102, 'dname': '멀티미디어학과', 'loc': '2호관'},
 {'deptno': 201, 'dname': '전자공학과', 'loc': '3호관'},
 {'deptno': 202, 'dname': '기계공학과', 'loc': '4호관'}]

### 데이터프레임으로 변환

In [29]:
df = DataFrame(result)
df.set_index('deptno', inplace=True)
df

,dname,loc
deptno,,
101,컴퓨터공학과,1호관
102,멀티미디어학과,2호관
201,전자공학과,3호관
202,기계공학과,4호관
